In [1]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from matplotlib import pyplot as plt

In [2]:
def preprocess(file_path):
    
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    
    img = tf.image.resize(img, (100,100))
    img = img / 255.0
    
    return img

In [3]:
def verify(model, detection_threshold, verification_threshold):
    max_verification_folders=10
    for i in range(1, max_verification_folders + 1):
        folder_path = os.path.join('application_data', f'verification_images{i}')
        if not os.path.exists(folder_path):
            break
        
        results = []
        for image in os.listdir(folder_path):
            if image.endswith('.jpg'):
                input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
                validation_img = preprocess(os.path.join(folder_path, image))
                
                result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
                results.append(result)
        
        detection = np.sum(np.array(results) > detection_threshold)
        
        verification = detection / len(os.listdir(folder_path)) 
        verified = verification > verification_threshold
        
        print(verified)
        
        if verified:
            return results, verified, i
    
    return None, False, None

In [4]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
       
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [5]:
facetracker = load_model('facetracker.h5')
facetracker.summary()

2023-10-26 10:07:03.252963: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-10-26 10:07:03.252997: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-26 10:07:03.253005: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-26 10:07:03.253061: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-26 10:07:03.253086: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 120, 120, 3)]        0         []                            
                                                                                                  
 vgg16 (Functional)          (None, None, None, 512)      1471468   ['input_5[0][0]']             
                                                          8                                       
                                                                                                  
 global_max_pooling2d_2 (Gl  (None, 512)                  0         ['vgg16[0][0]']               
 obalMaxPooling2D)                                                                                
                                                                                            

In [6]:
# Reload model 
siamese_model = tf.keras.models.load_model('siamesemodelv3.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 3896044   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     

In [7]:
# experiments with multiple folders
results, verified, i= verify(siamese_model, 0.9, 0.9)
if i==1:
    print(verified, 'Adam')
elif i==2:
    print(verified, 'Archmagos Belisaria')
elif i==3:
    print(verified, 'Aaditya')
print(results, verified, i)

None False None


In [9]:
#Multiple
res=''

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[100:100+600,650:650+600 :]
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        results, verified, i = verify(siamese_model, 0.5, 0.5)
        if i==1:
            res='Adam'
        elif i==2:
            res='Archmagos Belisaria'
        elif i==3:
            res='Aaditya Paliwal'

        
        print(verified)
    
    #bounding_box
    crop_size = 600
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120, 120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255, 0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [crop_size, crop_size]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [crop_size, crop_size]).astype(int)), 
                            (0, 255, 0), 3)
    cv2.imshow('Side Face Detection', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 13ms/step
False
1/1 [==============================] - 0s 13ms/step
False
1/1 [==============================] - 0s 13ms/step
False
1/1 [==============================] - 0s 12ms/step
False
1/1 [==============================] - 0s 12ms/step
False
1/1 [==============================] - 0s 13ms/step
False
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step
False
False
1/1 [==============================] - 0s 13ms/step


In [8]:
res

'Aaditya Paliwal'